In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import json
import os
from os.path import exists
import time
import sys
sys.path.insert(1, '../../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
_data_ = json.loads(open('../../../data/2024_vast/MC1/mc1.json').read())

In [ ]:
_json_repr_ = rt.jsonRepr(_data_)
_df_, _relates_, _labels_ = _json_repr_.starPathGraphDataFrame()
g   = rt.createNetworkXGraph(_df_, _relates_)
pos = rt.hyperTreeLayout(g)
#rt.interactiveGraphLayout(_df_, ln_params={'relationships':_relates_, 'node_labels':_labels_, 'pos':pos}, w=800, h=600)

In [ ]:
_lu_ = {'sort':[]}
for i in range(len(_data_['nodes'])):    
    as_list = list(_data_['nodes'][i].keys())
    as_list.sort()
    _lu_['sort'].append('-'.join(as_list))
rt.co_mgr.optimizeCategoricalColors(set(_lu_['sort']))
rt.histogram(pd.DataFrame(_lu_), bin_by='sort', color_by='sort', w=384, h=64)

In [ ]:
_lu_ = {'links':[]}
for i in range(len(_data_['links'])):    
    as_list = list(_data_['links'][i].keys())
    as_list.sort()
    _lu_['links'].append('-'.join(as_list))
rt.co_mgr.optimizeCategoricalColors(set(_lu_['links']))
rt.histogram(pd.DataFrame(_lu_), bin_by='links', color_by='links', w=768, h=64)